## Import packages

In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline
from tqdm import tqdm

sys.path.insert(0, os.path.dirname(os.path.abspath('..')))
from mass_automation.experiment import Experiment, Spectrum
from mass_automation.formula import Formula
from mass_automation.formula.check_formula import del_isotopologues
from research.deisotoping.train_models.generate_mixtures import create_substance_list, open_molecules


C:\Users\user\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\user\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


This notebook is for defining average mass difference between isotopologues and minimal distance evaluation (these parameters are important for well-performed deisotoping procedure)

### 1) Investigation of the average mass difference (corrected neutron mass) and  **delta** parameter  

<b> !! Path to txt-file with formulas from PubChem is located in *data/pubchem_formulas/0.1_subsample.merged.subsampled_honestly.txt* !! </b>

In [2]:
path_to_pubchem = os.path.join(os.path.dirname(os.path.abspath('..')), 'data',
                               'pubchem_formulas', '0.1_subsample.merged.subsampled_honestly.txt')

In [3]:
objects = open_molecules(path_to_pubchem)  # This function opens file with saved molecular formulas from pubchem
substance_list = create_substance_list(objects, size=500)  # This function generates a list of formulas with size 500

# This is mass error for augmentation of isotopologue distributions. We did it to make data more realistic
dist_error = 2.6*10**(-4) 

In [ ]:
# Here we create a list with all possible mass differences between isotopologues of substances in substance_list
total_diffs = np.array([])
for substance in tqdm(substance_list):
    formula = Formula(substance['formula'])  # Create Formula object
    masses, peaks = formula.isodistribution() # Create isotopic distribution out of formula (direct problem)
    masses, peaks = del_isotopologues(masses, peaks) # Delete fine isotopic structure
    for i in range(len(masses)):
        masses[i] = np.random.uniform(masses[i] - dist_error, masses[i] + dist_error)  # Augmentation with mass error
    diffs =np.diff(masses)
    total_diffs = np.append(total_diffs, diffs) 

 11%|████████▉                                                                        | 55/500 [00:20<00:45,  9.75it/s]

P.S. If the code in the cell above is being executed slow, try to reduce size in *substance_list* and restart the kernel

In [ ]:
# Density estimation 
kde = sm.nonparametric.KDEUnivariate(total_diffs)
kde.fit(kernel='epa', bw='scott', fft=False)  # Estimate the densities

plot resulted distribution:

In [ ]:
hfont = {'fontname':'DejaVu Sans'}
fig = plt.figure(figsize=(5.817, 3.932))
ax = fig.add_subplot(111)

lines = ax.hist(total_diffs, bins='auto', label="Histogram")
ax.plot(kde.support, kde.density, lw=3, label="KDE from samples", zorder=10)
ax.legend(loc="best")
ax.set_xlim(0.990, 1.006)
xticks = [0.990 + 0.002*i for i in range(9)]
ax.set_xticks(xticks)
ax.set_xlabel('mass difference, Da', **hfont)
ax.set_ylabel('counts', **hfont)
#fig.savefig('Average_mass_on_organics_kde.png', dpi=300, bbox_inches='tight')

We see an interesting distribution. Two clusters are obtained, that is the reason why ML-deisotoping is better. Model should be more certain in merging two peaks into one isotopic distribution if mass difference between them is in the center of one of the clusters. We can't recognize these patterns using only linear classification because we have only two separating hyperplanes (for 1D data hyperplanes are just points)

Anyway, we got parameters:

Average mass difference = <span style="color:blue">0.998</span> <br>
Delta = <span style="color:blue">0.007</span>

### 2) Optimal min_distance investigation

Peak picking function in MEDUSA has a parameter min_distance, which is the minimal mass difference between peaks in spectra, that are gonna be processed. In deisiotoping we try to deisotope peaks from fine isotopic structure, so we should examine, what is the possible maximal mass difference between aggregatted isotopic variants.

We will use substances from validation dataset, as their isotopologues have rich fine isotopic structure

In [ ]:
list_of_formulas = ['C7H5O3',
 'C13H10N',
 'C7H7O3S',
 'C15H21Ni2O6',
 'C7H10NNiO2',
 'C9H13N2NiO2',
 'C27H36N2PdC2H6C2N2Cl',
 'C27H36N2PdC5H5NCH3CNCl',
 'C27H36N2PdC5H5NCl',
 'C27H36N2PdCH3CNCl',
 'C10H14FeO4',
 'C10H14MnO4',
 'C25H35Mn2O10',
 'C19H18P',
 'C25H23P2',
 'C19H42N',
 'C10H16ClN2Pd2',
 'C5H8NPd',
 'C6H10ClPd2',
 'C7H11N2Pd',
 'C8H13ClNPd2',
 'C27H36N2H',
 'C57H52N',
 'C9H11N2',
 'C7H11N2',
 'C18H16OP',
 'C8H15N2',
 'C10H14CuNaO4',
 'C20H28Cu2NaO8',
 'C26H29N2Ni',
 'C28H32N3Ni',
 'C10H6Cu7N8',
 'C11H6Cu8N9',
 'C5H6Cu2N3',
 'C6H6Cu3N4',
 'C7H6Cu4N5',
 'C8H6Cu5N6',
 'C9H6Cu6N7',
 'C18H15NaOP',
 'C18H16OP',
 'C36H30NaO2P2',
 'C32H32NO4',
 'C16H36N',
 'C28H24N',
 'C30H28N',
 'C40H40IrN4',
 'C23H22N',
 'C28H30N2O3Na',
 'C28H31N2O3',
 'C4H6BrN2Ni']

Here we cluster peaks to find aggregated isotopic variants and get list with maximal mass differences between aggregated isotopic variants for every compound in above list

In [ ]:
edge_dist_max_list = []
for formula in list_of_formulas:
    formula = Formula(formula)
    masses, peaks = formula.isodistribution()
    round_masses = np.round(masses, 1)
    unique_masses = np.unique(round_masses)
    mass_dict = {}
    for mass in unique_masses:
        isotopic_variants_masses = masses[round_masses == mass]
        isotopic_variants_peaks = peaks[round_masses == mass]
        new_is_var_masses = isotopic_variants_masses[isotopic_variants_peaks > 0.001*isotopic_variants_peaks.max()]
        edge_dist = new_is_var_masses.max() - new_is_var_masses.min()
        mass_dict[mass] = edge_dist
    edge_dist_max = max(mass_dict.values())
    edge_dist_max_list.append(edge_dist_max)

In [ ]:
# Plotting the histogram
hfont = {'fontname':'DejaVu Sans'}
plt.figure(figsize=(5.817, 3.932))
plt.hist(edge_dist_max_list, bins=25)
xticks = [0.00, 0.01, 0.02, 0.03, 0.04]
plt.xticks(xticks)
plt.xlabel('maximal distance between aggregated isotopic variants, Da', **hfont)
plt.ylabel('counts', **hfont)
#plt.savefig('Max_dist_between_fine_structures.png', dpi=300, bbox_inches='tight')
plt.show()

Maximal distance (or mass difference) is 0.02. However, this is the estimation above, as 99% of isotopologues have smaller maximal distance. The isotopic structure is symmetric to the most intense peak, so *min_distance* parameter should be 0.01